In [ ]:
# Dependency
import pytaxonkit as ptax
import pandas as pd
import os
import math

In [ ]:
# INPUT
unique_taxa=set(pd.read_csv(snakemake.input[0],sep="\t")["Taxa"])
lineage_df=ptax.lineage(unique_taxa,data_dir=os.path.dirname(snakemake.input[1]))

In [ ]:
#Fix code taxid issue
result=[]
for index, row in lineage_df.iterrows():
    if row["TaxID"]!=row["Code"]:
        tmp=row
        tmp["TaxID"]=tmp["Code"]
        result.append(tmp)
lineage_df=lineage_df.append(result)
#TODO change to concat

In [ ]:
#reformat_df
result=[]
for index, row in lineage_df.iterrows():
    x=row["FullLineageRanks"].split(";")
    y=[int(i) for i in row["FullLineageTaxIDs"].split(";")]
    z=dict(zip(x, y))
    z.pop('no rank', None)
    z["TaxID"]=row["TaxID"]
    z["Name"]=row["Name"]
    result.append(z)
result=pd.DataFrame(data=result)
relevant_columns=["superkingdom","clade","phylum","class","order","family","genus","species","strain","TaxID","Name"]
result=result[relevant_columns]
result.rename(columns={"TaxID": "Taxa"},inplace=True)

In [ ]:
# OUTPUT
with open(snakemake.output[0],'w') as out_f:
    result.to_csv(out_f,sep="\t", index=False)